In [ ]:
import numpy as np
import pandas as pd

datadir = '/kaggle/input/multimodal-classification-2021-mi203'

import os

# TRAIN DATA

train_data = pd.read_csv(os.path.join(datadir,'data/data_train.csv'), delimiter=',', nrows = None)
data = np.array(train_data)

y_train = data[:,-1].astype('int32')

audio_train = data[:, 1:-1]

img_list_train = train_data['IMAGE']

# TEST DATA

test_data = pd.read_csv(os.path.join(datadir,'data/data_test_novt.csv'), delimiter=',', nrows = None)
data = np.array(test_data)

audio_test = np.array(data[:, 1:], dtype='float64')

img_list_test = test_data['IMAGE']



In [ ]:
print(audio_train.shape)

In [ ]:
#====================================================================
# 1. Data preprocessing: Scaling and separation of train and test sets

import numpy as np
import matplotlib.pyplot as plt
import random
import optuna

from sklearn.model_selection import train_test_split
from sklearn import preprocessing

# Normalisation des données audio
scaler = preprocessing.StandardScaler().fit(audio_train)
audio_train_scaled = scaler.transform(audio_train)

# Séparation des données en apprentissage et test
Xn, Xv, yn, yv = train_test_split(audio_train_scaled, y_train,random_state=42,test_size=0.2)




In [ ]:
#====================================================================
# 2. Hyperparameter optimization

# -----------------------------------------------------------------------------------------------------------------------------------------------
# Attempt to optimize hyperparameters "manually", checking the precision on train and test sets for different values of C (regularity of the SVM)
# -----------------------------------------------------------------------------------------------------------------------------------------------

from sklearn import svm
C_test = np.logspace(-1,1.2,15)

for i in C_test:
    svc = svm.SVC(kernel='rbf', max_iter=-1, verbose = True, C=i)
    svc.fit(Xn, yn)
    score_train = svc.score(Xn, yn)
    score_test = svc.score(Xv, yv)
    print("C:" , i, " | Taux de reco = {:.2f}% / {:.2f}%".format(score_train*100, score_test*100)," | Rapport = {:.3f}".format(score_train/score_test))
    
# A good value apparently is C=2.6
# Good values must have high precision on both train and test sets (low variance), without a great difference between them (low bias)

In [ ]:
print("Taux de reco = {:.2f}% / {:.2f}%".format(score_train*100, score_test*100))
print(y_pred)

In [ ]:
#====================================================================
# 2. Final learning (SVM)

# -----------------------------------------------------------------------------
# Using the FULL data and the hyperparameters found in the optimization process
# -----------------------------------------------------------------------------

import numpy as np
import matplotlib.pyplot as plt
import random

# Normalisation des données audio
from sklearn import preprocessing

scaler = preprocessing.StandardScaler().fit(audio_train)

audio_train_scaled = scaler.transform(audio_train)

#====================================================================

from sklearn import svm

# Creating SVM with RBF kernel
svc = svm.SVC(kernel='rbf', max_iter=-1, verbose = True, C=2.6)

# Fitting SVM to all training data
svc.fit(audio_train_scaled, y_train)

# Prediction
y_pred = svc.predict(scaler.transform(audio_test))

In [ ]:
print(y_pred.shape)
print(y_pred)

In [ ]:
#====================================================================
# Création du ficher de soumission

submission = pd.DataFrame({'CLASS':y_pred})
submission=submission.reset_index()
submission = submission.rename(columns={'index': 'Id'})

#======================================================================
# Sauvegarde du fichier
submission.to_csv('audio_submission_v3.csv', index=False)
